# 2.Search Engine

In [57]:
import nltk
from nltk.stem import PorterStemmer

In [58]:
def text_mining(synopsis):
    tokens = nltk.word_tokenize(synopsis)                           #tokenization
    tokens = [word for word in tokens if word.isalpha()]            #remove punctuations and numbers
    stop_words = set(nltk.corpus.stopwords.words('english'))        #remove stopwords
    tokens = [word for word in tokens if not word.lower() in stop_words]       
    tokens = [PorterStemmer().stem(word) for word in tokens]        #stemming
    tokens = [str(hash(word)) for word in tokens]                   #consider the hash code of the words will be useful for the next queries
    return tokens

In [59]:
def vocab(text):
    for word in set(text):                                          #create the vocabuary using the synopsis 
        vocabulary.append(word)

In [96]:
description = {}
vocabulary = []
num_pag = 20
for j in range(0,num_pag):
    for i in range((50*j)+1, (50*(j+1))+1):
        file = open( "pages_tsv/pages_tsv/page_" + str(j+1) + "/anime_" + str(i)+ ".tsv", "r", encoding = "utf-8")       
        anime = file.read()                                             
        file.close()
        anime = anime.split("\t")
        synopsis = anime[24]
        synopsis = text_mining(synopsis)
        vocab(synopsis)
        doc = "".join("documento "+str(i))
        description[doc] = synopsis
        title = anime[14]

### 2.1. Conjunctive query

In [97]:
vocabulary = list(set(vocabulary))                    #create the vocabulatory with non repetitions of words
file = open("vocabulary.txt", "w", encoding = "utf-8")
for word in vocabulary:
    file.write(str(hash(word)) +"\n")                 #create the file .txt that maps each word to a term_id using tha hash
file.close()                                          #function of python

### 2.1.1) Create your index!

In [98]:
inverted_index = {}
for term_id in vocabulary:
    for i in description:
        if term_id in description[i]:
            if term_id not in inverted_index:
                inverted_index[term_id] =  [i]
            else:
                inverted_index[term_id] = inverted_index[term_id] + [i]


### 2.1.2) Execute the query

In [99]:
def search_word(query):
    d = []
    query = set(text_mining(query))
    for word in query:
        d.append(inverted_index[word])
    print(d)
    for i in range(len(d)-1):
        s = set(d[i]).intersection(set(d[i+1]))
    return  s
        

In [100]:
query = "saiyan race"
search_word(query)

[['documento 91', 'documento 100', 'documento 119', 'documento 141', 'documento 143', 'documento 147', 'documento 173', 'documento 184', 'documento 235', 'documento 236', 'documento 240', 'documento 260', 'documento 271', 'documento 297', 'documento 314', 'documento 333', 'documento 365', 'documento 371', 'documento 378', 'documento 391', 'documento 397', 'documento 401', 'documento 452', 'documento 457', 'documento 494', 'documento 501', 'documento 504', 'documento 515', 'documento 518', 'documento 529', 'documento 560', 'documento 603', 'documento 608', 'documento 613', 'documento 661', 'documento 680', 'documento 694', 'documento 726', 'documento 732', 'documento 749', 'documento 750', 'documento 774', 'documento 798', 'documento 802', 'documento 811', 'documento 831', 'documento 898', 'documento 910', 'documento 928'], ['documento 365', 'documento 401']]


{'documento 365', 'documento 401'}